#2 - Cleaning the Data
==================

In order to prepare my final data file, the individual frames had to be cleaned and concatenated:

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt

`Cleaning pollution data:`
-------------------------------------

In order to clean the data, I used several steps:

**1.** I dropped the first column, and removed the '24:00' part of my date strings.

**2.** I converted the data type of the three particle columns (**NO**, **NO2**, **NOX**) to float, using **coerce** to convert non-numeric values to NaN.

**3.** I dealt with missing values by using the following strategy:

* If the first two cells are **NaN**, their value will be **equal to the column's mean**. 


* If the last cell is **NaN**, it will be **equal to the column's current mean**.


* For any other cell:

    * If the next cell is **NaN**, the value will be **equal to the mean of the two previous cells**.
    
    * Otherwise, the value will be **equal to the mean of the previous and the next cells**.
    
The new file will be saved as **'pollution_data_cleaned.csv'**.

In [2]:
def load_pollution_data(filename):
    return pd.read_csv(filename)  #load file

In [3]:
def clean_pollution_data(df, filename):
    #Remove the generated index column
    df.drop(columns=df.columns[0], axis=1, inplace=True)   
        
    #Remove 24:00 from the day strings
    df['date'] = df['date'].str.replace("24:00 ", '')
    
    #Convert particle columns to numeric values
    df['NO'] = pd.to_numeric(df['NO'], errors='coerce')
    df['NO2'] = pd.to_numeric(df['NO2'], errors='coerce')
    df['NOX'] = pd.to_numeric(df['NOX'], errors='coerce')    
    
    #Fill missing values according to the formula explained above
    for column in df[['NO', 'NO2', 'NOX']]:        
        for i in range(len(df[column])):            
            curr = df.at[i, column]
            #If first or second cell
            if i == 0 or i == 1:                
                #Ensure first values are not none, else fill them with the column's mean                
                if pd.isnull(curr):
                    df.at[i, column] = df[column].mean()
                prev_2 = df.at[0, column]
                prev = df.at[1, column]
            #If last cell
            if i == len(df[column]):
                if pd.isnull(curr):
                    df.at[i, column] = df[column].mean()            
            #In the case of other cells
            else:                
                if pd.isnull(curr):
                    #Check if the next value is NaN
                    next = df.at[i+1, column]
                    if pd.isnull(next):                        
                        #If so, the value will be equal to the mean of the previous two values
                        df.at[i, column] = ((prev_2 + prev)/2)
                    else:
                        #If not, the value will be the mean of the previous and the next
                        df.at[i, column] = ((prev + next)/2)
                prev_2 = prev
                prev = df.at[i, column] 
    
    df.to_csv('{0}.csv'.format(filename))
    print("Cleaned dataframe created successfully.")
    return df

In [4]:
poll_filename = 'pollution_data_raw.csv'
poll_filename_new = 'pollution_data_cleaned'
df_poll = load_pollution_data(poll_filename)
df_poll_clean = clean_pollution_data(df_poll, poll_filename_new)

Cleaned dataframe created successfully.


In [5]:
df_poll_clean.dtypes

date        object
station     object
NO         float64
NO2        float64
NOX        float64
dtype: object

**The cleaned pollution dataframe:**

In [6]:
df_poll_clean

,date,station,NO,NO2,NOX
0,01/01/2016,Ariel,1.5,3.8,6.1
1,02/01/2016,Ariel,1.1,4.9,6.7
2,03/01/2016,Ariel,1.4,10.8,13.0
3,04/01/2016,Ariel,1.8,9.7,12.5
4,05/01/2016,Ariel,2.1,11.8,15.1
...,...,...,...,...,...
23746,27/12/2020,Tel Aviv,11.7,28.0,39.7
23747,28/12/2020,Tel Aviv,11.5,28.1,39.6
23748,29/12/2020,Tel Aviv,31.2,40.2,71.4
23749,30/12/2020,Tel Aviv,19.3,44.7,64.1


`Cleaning precipitation data:`
-------------------------------------

Here, I had different issues to deal with:

**1.** Since the text was in Hebrew, I had to encode the data in the ISO-8859-8 format, which supports the Hebrew alphabet.

**2.** I dropped the irrelevant columns - such as the station number.

**3.** I translated the column names to English, and then converted the station names from Hebrew to the exact English names used in the df_poll dataframe by using a dictionary and mapping the values after cleaning auto-generated white spaces in the Hebrew strings.

The new file will be saved as **'precipitation_data_cleaned.csv'**.

In [7]:
def load_precipitation_data(filename):
    return pd.read_csv(filename, encoding = "ISO-8859-8")  #load file with Hebrew encoding

In [8]:
def clean_precipitation_data(df, filename):
    #Drop irrelevant columns
    df.drop(['מספר תחנה', 'קוד גשם יומי()'], axis=1, inplace = True)

    #Change column names
    df.rename(columns={'שם תחנה': "station", 'תאריך': "date", 'כמות גשם יומית(מ"מ)': "precipitation"}, inplace = True)

    #Translate station names
    stations = {"אריאל מכללה": 'Ariel',
                "אשדוד נמל": 'Ashdod',
                "באר שבע": 'Beer Sheva',
                "חדרה תחנת הכח": 'Hadera',
                "חיפה בתי זיקוק": 'Haifa',
                "ירושלים מרכז": 'Jerusalem',
                "כפר מנחם, כפר מנחם": 'Kfar Menachem',
                "אשחר": 'Karmiel',
                "עפולה ניר העמק": 'Afula',
                "ערד": 'Arad',
                "גת": 'Kiryat Gat',
                "ראשון לציון": 'Rishon LeZion',
                "תל-אביב חוף": 'Tel Aviv'}

    df['station'] = df['station'].str.strip()
    df['station'] = df['station'].map(stations)
    
    df['precipitation'] = pd.to_numeric(df['precipitation'], errors='coerce')
    
    df.to_csv('{0}.csv'.format(filename))
    print("Cleaned dataframe created successfully.")    
    return df    

In [9]:
prec_filename = 'precipitation_data_raw.csv'
prec_filename_new = 'precipitation_data_cleaned'
df_prec = load_precipitation_data(prec_filename)
df_prec_clean = clean_precipitation_data(df_prec, prec_filename_new)

Cleaned dataframe created successfully.


**The cleaned precipitation dataframe:**

In [10]:
df_prec_clean

,station,date,precipitation
0,Afula,01/01/2016,10.3
1,Afula,02/01/2016,8.3
2,Afula,03/01/2016,0.9
3,Afula,07/01/2016,29.5
4,Afula,08/01/2016,39.9
...,...,...,...
3729,Tel Aviv,15/12/2020,19.9
3730,Tel Aviv,16/12/2020,58.2
3731,Tel Aviv,17/12/2020,8.5
3732,Tel Aviv,23/12/2020,5.3


`Concatenating the data:`
-------------------------------------

In order to concatenate the data, I had to use the **df.merge** function, fitting the relevant precipitation data in **df_prec** into the correct locations in the in the **df_poll** dataframe based on the date and station. Since precipitation data is only given in 3734 records, I assumed that the other days had no recorded precipitation, and filled them with 0s.

The new, and final file will be saved as **'data_final.csv'**.

In [11]:
def concat_data(df_poll, df_prec, filename):
    merged_df = df_poll.merge(df_prec, how = 'left', on = ['date', 'station'])
    merged_df['precipitation'] = merged_df['precipitation'].fillna(0)
    
    merged_df.to_csv('{0}.csv'.format(filename))
    print("Concatenated dataframe created successfully.")    
    return merged_df  

In [12]:
filename_final = 'data_final'
df_final = concat_data(df_poll, df_prec, filename_final)

Concatenated dataframe created successfully.


**The final dataframe, which will be used in the next steps:**

In [13]:
df_final

,date,station,NO,NO2,NOX,precipitation
0,01/01/2016,Ariel,1.5,3.8,6.1,13.6
1,02/01/2016,Ariel,1.1,4.9,6.7,7.4
2,03/01/2016,Ariel,1.4,10.8,13.0,9.0
3,04/01/2016,Ariel,1.8,9.7,12.5,0.0
4,05/01/2016,Ariel,2.1,11.8,15.1,0.0
...,...,...,...,...,...,...
23746,27/12/2020,Tel Aviv,11.7,28.0,39.7,0.0
23747,28/12/2020,Tel Aviv,11.5,28.1,39.6,0.0
23748,29/12/2020,Tel Aviv,31.2,40.2,71.4,0.0
23749,30/12/2020,Tel Aviv,19.3,44.7,64.1,0.0
